In [10]:
from calendar import monthrange
import requests

def getDataCentral(path, filename, ext = ".xls"):
    '''
    Funcion para descargar data del Banco central
    
    path: ruta del documento a descargar
    filename: nombre del archivo a grabar la data descargada
    ext: la extencion del archivo. Por lo general los del banco central son '.xls' 
    
    TODO: Falta poner que se descarguen en la carpeta de data
    
    '''
    resp = requests.get(path)
    
    with open(filename + ext, "wb") as output:
        output.write(resp.content)

In [11]:
documentos = {"PIB": "https://gdc.bancentral.gov.do/Common/public/estad%C3%ADsticas/sector-real/documents/pib_gasto_2007.xls",
              "IPC": "https://gdc.bancentral.gov.do/Common/public/estad%C3%ADsticas/precios/documents/ipc_base_2010.xls",
              "TPM": "https://gdc.bancentral.gov.do/Common/public/estad%C3%ADsticas/sector-monetario-y-financiero/documents/Serie_TPM.xlsx",
              "tasa_dolar": "https://gdc.bancentral.gov.do/Common/public/estad%C3%ADsticas/mercado-cambiario//documents//TASA_DOLAR_REFERENCIA_MC.XLS",
              "tasa_cambio_todas": "https://gdc.bancentral.gov.do/Common/public/estad%C3%ADsticas/mercado-cambiario//documents//TASAS_CONVERTIBLES_OTRAS_MONEDAS.xls",
              "agregados_monetarios": "https://gdc.bancentral.gov.do/Common/public/estad%C3%ADsticas/sector-monetario-y-financiero/documents/agregados_monetarios.xls",
              "reservas_internacionales": "https://gdc.bancentral.gov.do/Common/public/estad%C3%ADsticas/sector-externo/documents/reservas_internacionales.xls"}

In [12]:
for filename, path in documentos.items():
    getDataCentral(path, filename)